In this simple example we configure our `thermocalc` 
model entirely from the script without utilising 
configuration files. 

We walk through various features of the `tawnycalc` 
configuration, execution, and data extraction. 

In [1]:
import tawnycalc as tc

In [2]:
# To start with a clean context, set `scripts_dir=None`
context = tc.Context(scripts_dir=None)
context.print_prefs()

calcmode   : 1
scriptfile : zundrw
dataset    : None


In [3]:
# Set the required dataset on the `prefs` dictionary.
context.prefs["dataset"] = 62

In [4]:
# Configure some parameters directly on the `script` dictionary.
help(tc.Context.script)

Help on property:

    The `script` dictionary contains your `thermocalc` model configuration. 
    This dictionary will be populated with the data from your `thermocalc` 
    scripts where the `scripts_dir` parameter is passed when constructing
    your `Context`.  Otherwise it will be empty and you will be required 
    to sufficiently populate it.  In either case, you may add, modify and
    delete values as necessary before calling `execute`. 
    
    Key/Value pairs will be directly printed to form standard `thermocalc`
    input.  For example 
    
    >>> context.script[       "axfile"] = "mb50NCKFMASHTO"
    >>> context.script[        "which"] = "chl bi pa ep ru chl g ilm sph"
    >>> context.script[     "inexcess"] = "mu q H2O"
    >>> context.script[       "dogmin"] = "yes 0"
    
    is equivalent to
    
        axfile   mb50NCKFMASHTO
        which    chl bi pa ep ru chl g ilm sph
        inexcess mu q H2O
        dogmin   yes 0
    
    To form repeat keyword entries, se

In [5]:
context.script[       "axfile"] = "mb50NCKFMASHTO"
context.script[        "which"] = "chl bi pa ep ru chl g ilm sph"
context.script[     "inexcess"] = "mu q H2O"
context.script[       "dogmin"] = "yes 0"
context.script[    "setmaxvar"] = "6"
context.script[   "setPwindow"] = "11 11"
context.script[   "setTwindow"] = "600 600"
context.script["pseudosection"] = "yes"
context.script[   "samecoding"] = ["mu pa", "sp mt"]
context.script[      "ptguess"] = "8.0 572"

In [6]:
# Add 'rbi' section.
# We have specific `rbi` object that we use to achieve this. 
# Let's create rbi object, noting how we provide the oxide list at construction time:
rbi_sec = tc.rbi(oxides="H2O SiO2 Al2O3 CaO MgO FeO K2O Na2O TiO2 O")

In [7]:
# Now we need to add the individual phases.
# We will use the `add_phase` method to achieve this. 
# We supply the `phase` name, `mode` data, and finally
# `oxides` as a string or list
help(rbi_sec.add_phase)

Help on method add_phase in module tawnycalc.data_objects:

add_phase(phase, mode, oxides) method of tawnycalc.data_objects.rbi instance
    Add a phase to the rbi table. 
    
    Params
    ------
    phase:  str
        Name of phase to add. 
    mode: str, float
        Mode/proportion of phase.
    oxides: str,list
        Proportion of each oxide for phase.



In [8]:
rbi_sec.add_phase(  "g",  "0.0"     , "       0    3.000000    0.984049    0.230147    0.395429    2.406326           0           0           0    0.015951")
rbi_sec.add_phase( "mu",  "0.159799", "1.000000    3.102219    1.393080    0.004483    0.058473    0.053149    0.386538    0.111220           0    0.002460")
rbi_sec.add_phase( "pa",  "0.006915", "       1    2.984010    1.505704    0.019628    0.001993    0.002589    0.033770    0.456416           0    0.000473")
rbi_sec.add_phase( "bi",  "0.090237", "0.932353    2.842048    0.636750           0    1.244448    1.572357    0.500000           0    0.067647    0.021202")
rbi_sec.add_phase("ilm",  "0.004242", "       0           0           0           0    0.011126    1.032649           0           0    0.956225    0.043775")
rbi_sec.add_phase(  "q",  "0.240992", "       0           1           0           0           0           0           0           0           0           0")
rbi_sec.add_phase("H2O",  "0.458826", "       1           0           0           0           0           0           0           0           0           0")

In [9]:
# Let's have a look at the object
rbi_sec

               H2O       SiO2      Al2O3     CaO       MgO       FeO       K2O       Na2O      TiO2      O
g    0.0       0         3.000000  0.984049  0.230147  0.395429  2.406326  0         0         0         0.015951
mu   0.159799  1.000000  3.102219  1.393080  0.004483  0.058473  0.053149  0.386538  0.111220  0         0.002460
pa   0.006915  1         2.984010  1.505704  0.019628  0.001993  0.002589  0.033770  0.456416  0         0.000473
bi   0.090237  0.932353  2.842048  0.636750  0         1.244448  1.572357  0.500000  0         0.067647  0.021202
ilm  0.004242  0         0         0         0         0.011126  1.032649  0         0         0.956225  0.043775
q    0.240992  0         1         0         0         0         0         0         0         0         0
H2O  0.458826  1         0         0         0         0         0         0         0         0         0

In [10]:
# You can also access individual values directly
rbi_sec["g"]["Al2O3"]

'0.984049'

In [11]:
# Values can also be modified
rbi_sec["g"]["Al2O3"] = 0.90
# Print again to confirm:
rbi_sec["g"]["Al2O3"]

0.9

In [12]:
# A point of subtlety... you may be noticed 
# above in the first instance we were returned
# '0.984049' which is actually a string, and 
# in the second instance we were returned 0.9,
# which is a float. Generally, you can use strings
# and their equivalent float interchangeably.

In [13]:
# Let's set that value back now
rbi_sec["g"]["Al2O3"] = 0.984049

In [14]:
# Ok, we have successfully created the `rbi` object, 
# but don't forget that it must be set on the `script` for
# it to take effect:
context.script["rbi"] = rbi_sec

In [15]:
# Now create xyzguess section.  
# We use the `xyz` object this time. 
# It is really just a Python `OrderedDict`.
# First create a dictionary to record values
xyzguess = tc.xyz()
# Now add values
xyzguess[    "x(g)"] = "0.885508"
xyzguess[    "z(g)"] = "0.211255"
xyzguess[    "f(g)"] = "0.070810"
xyzguess[   "x(bi)"] = "0.579607"
xyzguess[   "y(bi)"] = "0.140313"
xyzguess[   "f(bi)"] = "0.126258"
xyzguess[   "t(bi)"] = "0.072042"
xyzguess[   "Q(bi)"] = "0.121282"
xyzguess[   "x(mu)"] = "0.461998"
xyzguess[   "y(mu)"] = "0.944788"
xyzguess[   "f(mu)"] = "0.009495"
xyzguess[   "n(mu)"] = "0.253509"
xyzguess[   "c(mu)"] = "0.016160"
xyzguess[   "x(pa)"] = "0.461998"
xyzguess[   "y(pa)"] = "0.996868"
xyzguess[   "f(pa)"] = "0.002376"
xyzguess[   "n(pa)"] = "0.955475"
xyzguess[   "c(pa)"] = "0.038175"
xyzguess[   "x(ma)"] = "0.461998"
xyzguess[   "y(ma)"] = "0.964236"
xyzguess[   "f(ma)"] = "0.009701"
xyzguess[   "n(ma)"] = "0.054672"
xyzguess[   "c(ma)"] = "0.944410"
xyzguess[  "x(chl)"] = "0.473331"
xyzguess[  "y(chl)"] = "0.520964"
xyzguess[  "f(chl)"] = "0.248348"
xyzguess["QAl(chl)"] = "0.230540  range -1.000 1.000"
xyzguess[ "Q1(chl)"] = "0.077845  range -1.000 1.000"
xyzguess[ "Q4(chl)"] = "0.092301  range -1.000 1.000"
xyzguess[   "f(ep)"] = "0.264635"
xyzguess[   "Q(ep)"] = "0.250072  range  0.000 0.500"
xyzguess[  "i(ilm)"] = "0.829808"
xyzguess[  "g(ilm)"] = "0.017481"
xyzguess[  "Q(ilm)"] = "0.760053  range -0.990 0.990"
xyzguess[  "x(mt1)"] = "0.762081"
xyzguess[  "Q(mt1)"] = "0.808179"

In [16]:
# Again, don't forget to set on the script:
context.script["xyzguess"] = xyzguess

In [17]:
# Now we have a complete configuration, let's execute. 
results = context.execute(print_output=True)

THERMOCALC 3.50 (Free Pascal version)

the summary output is in the file, "tc-wlxlkt-o.txt"
other (eg drawpd) output is in the file, "tc-wlxlkt-dr.txt"
details of calc results is in the file, "tc-wlxlkt-ic.txt"
initial tables are in the file, "tc-wlxlkt-it.txt"
csv format in the file, "tc-wlxlkt-csv.txt"
more csv format in the file, "tc-wlxlkt-csv2.txt"

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

THERMOCALC 3.50 running at 14.32 on Tue 10 Mar,2020
using tc-ds62.txt produced at 20.08 on Mon 6 Feb,2012
with axfile tc-mb50NCKFMASHTO.txt and scriptfile tc-wlxlkt.txt

L  hb  aug  dio  opx  g  ol  ksp  plc  pli  sp  ilm  ilmm  ep  bi  mu
chl  ru  sph  q  an  ab  H2O
which: chl bi pa ep ru chl g ilm sph  (from script)

max variance of equilibria 6-variant (from script)

specification of PT window:
at fixed P = 11.00 kbar and T = 600.0íC (from script)
composition (from rbi script)
      H2O     SiO2    Al2O3      CaO      MgO      FeO      K2O     Na2O     T#
 

In [18]:
# Ok, we have a successful execution. 
# The resulting data is recorded in the returned
# dictionary object.  Execute the `print_keys()` method
# to see what is available. Note that keys prepended 
# with `output_` provide the raw output which may be useful 
# at times.
results.print_keys()

P
T
bulk_composition
modes
output_stderr
output_stdout
output_tc_ic
output_tc_log
phases
rbi
site_fractions
thermodynamic_properties
xyz


In [19]:
# Let's print the data. Note that most of the returned
# data objects are based in Python dictionaries. 
# You can therefore (where appropriate) also access 
# values directly.

In [20]:
results.P

11.0

In [21]:
results.T

600.0

In [22]:
results.bulk_composition

H2O    52.1705
SiO2   36.9434
Al2O3   4.3466
CaO     0.0129
MgO     1.7707
FeO     2.461
K2O     1.5886
Na2O    0.3156
TiO2    0.3454
O       0.0455

In [23]:
results.bulk_composition["H2O"]

52.1705

In [24]:
results.modes

g    0.0026
bi   0.0995
mu   0.1603
pa   0.0073
ru   0.0022
q    0.2503
H2O  0.4779

In [25]:
results.modes["g"]

0.0026

In [26]:
results.phases

'g bi mu pa ru q H2O (gmin, + ilm ep chl sph; in excess, mu q H2O)'

In [27]:
results.rbi

               H2O       SiO2      Al2O3     CaO       MgO       FeO       K2O       Na2O      TiO2      O
g    0.002576  0         3         0.984509  0.106355  0.410931  2.513696  0         0         0         0.015491
bi   0.099520  0.908899  2.829577  0.643344  0         1.177178  1.615455  0.500000  0         0.091101  0.027078
mu   0.160257  1         3.093818  1.401520  0.003133  0.051716  0.051425  0.383536  0.114898  0         0.003095
pa   0.007328  1.000000  2.989958  1.502832  0.013235  0.001703  0.002675  0.034180  0.459202  0         0.000593
ru   0.002157  0         0         0         0         0         0         0         0         1         0
q    0.250305  0         1         0         0         0         0         0         0         0         0
H2O  0.477857  1         0         0         0         0         0         0         0         0         0

In [28]:
results.rbi["mu"]["CaO"]

'0.003133'

In [29]:
results.site_fractions

g   xMgX     xFeX     xCaX     xAlY     xFe3Y
    0.13698  0.82757  0.03545  0.98451  0.01549
bi  xMgM3    xFeM3    xFe3M3   xTiM3    xAlM3    xMgM12   xFeM12   xSiT     xAlT     xOHV     xOV
    0.23132  0.50716  0.05416  0.09110  0.11627  0.47293  0.52707  0.41479  0.58521  0.90890  0.09110
mu  xKA      xNaA     xCaA     xMgM2A   xFeM2A   xAlM2A   xAlM2B   xFe3M2B  xSiT1    xAlT1
    0.76707  0.22980  0.00313  0.05172  0.04523  0.90305  0.99381  0.00619  0.54691  0.45309
pa  xKA      xNaA     xCaA     xMgM2A   xFeM2A   xAlM2A   xAlM2B   xFe3M2B  xSiT1    xAlT1
    0.06836  0.91840  0.01324  0.00170  0.00149  0.99681  0.99881  0.00119  0.49498  0.50502

In [30]:
results.site_fractions["bi"]["xAlM3"]

'0.11627'